In [1]:
from Bio import SeqIO
from Bio.SeqUtils import GC
import numpy as np
import glob
import pandas as pd

glob_AS = glob.glob('./ncbi_antismash/nf_output/*/')
genome_list,taxa_list,fragBGC,compBGC,totalBGC,seen = [],[],[],[],[],[]

for AS_path in glob_AS:
    genome_file = (AS_path.split('/')[3]).split('_')[0]
    genome_list.append(genome_file.split('.')[0])
    genome_path = "./ncbi_genomes/%s"%genome_file
    with open(genome_path) as f:
        first_line = f.readline()
        if 'TPA_asm:' in str(first_line):
            taxa = first_line.split(' ')[2]
        if 'uncultured' in str(first_line):
            taxa = first_line.split(' ')[2]
        if 'TPA_asm:' not in str(first_line) and 'uncultured' not in str(first_line):
            taxa = first_line.split(' ')[1]
        taxa_list.append(taxa)
    glob_BGCs = glob.glob('%s/*region*.gbk'%AS_path)
    if glob_BGCs:
        frag_count = 0
        comp_count = 0
        for BGC in glob_BGCs:
            input_handle = open(BGC,"r")
            for record in SeqIO.parse(input_handle,"genbank"):
                for feat in record.features:
                    if feat.type == 'cand_cluster':
                        if BGC not in seen:
                            seen.append(BGC)
                            if str(feat.qualifiers['contig_edge'][0]) == 'True':
                                frag_count += 1
                            else:
                                comp_count += 1
        fragBGC.append(frag_count)
        compBGC.append(comp_count)
        totalBGC.append(frag_count+comp_count)
    else:
        fragBGC.append(0)
        compBGC.append(0)
        totalBGC.append(0)
    

def get_draft_counts(query_genome):
    input_handle = open(query_genome,"r")
    node_count = 0
    gc_count = []
    for record in SeqIO.parse(input_handle,"fasta"):
        node_count += 1
        gc_count.append(GC(record.seq))
    gc_average = np.average(gc_count)
    return node_count,gc_average
    input_handle.close()

node_list,gc_list = [],[]
    
for genome in genome_list:
    node_count,gc_average = get_draft_counts('./ncbi_genomes/%s.fasta'%genome)
    node_list.append(node_count)
    gc_list.append(gc_average)

print(len(genome_list),len(taxa_list),len(node_list),len(gc_list),len(fragBGC),len(compBGC),len(totalBGC))

frames = {'GenomeID':genome_list,'Taxa':taxa_list,'Scaffold_count':node_list,
         'GC_content':gc_list,'Fragmented_BGCs':fragBGC,'Complete_BGCs':compBGC,
         'Total_BGCs':totalBGC}

metadata_df = pd.DataFrame(data=frames)

# metadata_df.to_csv('cyanobiome_metadata_df-TFL200507.tsv',sep='\t')

ModuleNotFoundError: No module named 'Bio'

In [2]:
metadata_df

NameError: name 'metadata_df' is not defined

In [71]:
from bs4 import BeautifulSoup
import re
import pandas as pd
# import sys
import csv

soup = BeautifulSoup(open('/home/jovyan/jupyterdata/tiago/dorrestein_nuno/elizabeth/ncbi_antismash/ACEY01/index.html'), 'html.parser')
# soup = BeautifulSoup(open('./ncbi_antismash/nf_output/AADV02.fasta_output_results/index.html'), 'html.parser')


# table = soup.find_all('table')


# parse header
cnames = []
for tx in soup.findAll('th'):
    cnames.append(tx.getText())

# parse rows
results = []  
for row in soup.findAll('tr'):
    tmp = []
    aux = row.findAll('td')
    for a in aux:
        tmp.append(a.string)
    results.append(tmp)

# with open(re.sub('html', 'tsv', sys.argv[1]), "w") as f:
#     writer = csv.writer(f, delimiter='\t')
#     results = [cnames]+results
#     writer.writerows(results)


index_df = pd.DataFrame(results)

In [72]:
index_df

,0,1,2,3,4,5,6
0,None,None,None,None,None,None,None
1,None,None,"109,348","120,248",2-methylisoborneol,terpene,100%
2,None,None,None,None,None,None,None
3,None,None,1,"13,124",Coelibactin,NRPS,36%
4,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...
106,None,None,"20,315","44,873",None,None,None
107,None,None,"13,627","39,134",Isorenieratene,terpene,100%
108,None,None,1,"1,880",None,None,None
109,None,None,1,"25,138",A54145,NRPS,5%


In [76]:
indexes_to_keep = []

for i,r in index_df.iterrows():
    if str(index_df[2].loc[i]) != 'None':
        indexes_to_keep.append(i)
        
index_df = index_df.loc[indexes_to_keep]

index_df = index_df.reset_index(drop=True)

index_df = index_df[:int(len(index_df)/2)]

index_df

,0,1,2,3,4,5,6
0,None,None,"109,348","120,248",2-methylisoborneol,terpene,100%
1,None,None,1,"13,124",Coelibactin,NRPS,36%
2,None,None,1,"3,781",Coelibactin,NRPS,18%
3,None,None,1,"20,529",Alkylresorcinol,t3pks,100%
4,None,None,"6,147","27,274",Terfestatin,other,23%
5,None,None,"33,080","73,967",None,None,None
6,None,None,"73,174","99,915",Hopene,terpene,100%
7,None,None,"10,199","32,913",SapB,lanthipeptide,100%
8,None,None,"51,535","74,028",Nogalamycin,t2pks,40%
9,None,None,1,"3,568",None,None,None


In [70]:
ls /home/jovyan/jupyterdata/tiago/dorrestein_nuno/elizabeth/ncbi_antismash/ACEY01/*region*gbk | wc -l

37
